# 簡単なモンテカルロ・シミュレーション
多数の粒子の相互作用によって生じている現象を確率の考え方を使って記述するというのが統計力学です。統計力学の問題が解析的に解けない場合には数値シミュレーションをする必要がありますが、モンテカルロ法はその代表例の一つです。古典系にも量子系（量子系の場合は鈴木トロッター分解などの手続きが必要）にも扱えますが、今回は古典系、その代表例のイジング模型でのモンテカルロ・シミュレーションを扱ってみたいと思います。

## 統計力学のおさらい
モンテカルロ法で使う統計力学の考え方を、ざっとおさらいしておきましょう。
